In [1]:
from preprocessing import *
from import_data import *
data_dir = 'DataCollection/outputs/'
ssvep_dir = data_dir + 'SSVEP/sess3/'
col_names = ['Fp1', 'O1', 'O2','C3','C4','P3','P4','Pz']
ssvep_files = get_files(ssvep_dir)

In [2]:
ssvep_files

['DataCollection/outputs/SSVEP/sess3/2023-03-10T14:47:20.913143_baseline.txt',
 'DataCollection/outputs/SSVEP/sess3/2023-03-10T14:47:20.913143_data.txt',
 'DataCollection/outputs/SSVEP/sess3/2023-03-10T14:47:20.913143_metadata.txt',
 'DataCollection/outputs/SSVEP/sess3/2023-03-10T14:55:08.474247_baseline.txt',
 'DataCollection/outputs/SSVEP/sess3/2023-03-10T14:55:08.474247_data.txt',
 'DataCollection/outputs/SSVEP/sess3/2023-03-10T14:55:08.474247_metadata.txt']

In [6]:
data = pd.read_csv(ssvep_files[1], header=None)

In [7]:
data

,0,1,2,3,4,5,6,7,8
0,282100.426963,5139.828125,-22348.011719,-3630.392578,-10867.999023,-15533.948242,1681.387573,-25343.906250,-23204.933594
1,282100.430963,5088.061523,-22358.919922,-3708.556641,-10813.975586,-15546.710938,1772.381592,-25314.111328,-23195.343750
2,282100.434963,5120.985840,-22289.941406,-3794.566162,-10805.101562,-15569.286133,1770.839355,-25259.662109,-23184.660156
3,282100.438963,5137.548340,-22317.748047,-3712.222412,-10830.269531,-15574.315430,1719.765625,-25309.259766,-23197.781250
4,282100.442963,5144.120117,-22365.937500,-3621.183838,-10857.740234,-15550.868164,1688.026123,-25345.515625,-23210.878906
...,...,...,...,...,...,...,...,...,...
24182,282197.067467,6162.241699,-22051.539062,-3340.065918,-10361.843750,-15940.101562,1931.995361,-25104.070312,-22735.054688
24183,282197.071467,6178.536133,-22020.603516,-3356.203857,-10340.050781,-15936.771484,1962.460815,-25088.871094,-22727.455078
24184,282197.075467,6164.365234,-21966.847656,-3428.511719,-10307.395508,-15947.456055,1998.715332,-25058.183594,-22713.955078
24185,282197.079467,6166.555664,-21994.072266,-3411.613770,-10320.850586,-15948.416992,1973.346069,-25067.906250,-22719.675781


In [8]:
Fs = 250
len(data) / Fs

96.748